In [1]:
import os

# Get the current working directory
current_directory = os.getcwd()
print("Current Directory:", current_directory)

# Move to the parent directory
parent_directory = os.path.dirname(current_directory)
os.chdir(parent_directory)

# Print the updated working directory
updated_directory = os.getcwd()
print("Updated Directory:", updated_directory)

Current Directory: /mnt/c/users/rjmay/code/research_ms/alt_distr
Updated Directory: /mnt/c/users/rjmay/code/research_ms


In [2]:
! source ../venv/bin/activate

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import math
import pickle
import time

from modules.constants import sample_sizes
sample_sizes  = np.array(sample_sizes)*2

In [4]:
sample_sizes

array([  100,   160,   250,   400,   630,  1000,  1600,  2500,  4000,
        6300, 10000])

In [145]:

dim_str = "15" # TODO change me 


In [146]:
# BER_values = {'beta_beta': 0.10937546875016925, 'paired_uniforms': .1, 'normal_beta': 0.0726617598018901,
#                'uniform_normal': 0.15350691875939732}

BER_values = {'gm': 0.13868461929442688, 'paired_uniforms': .1, 'normal_beta': 0.0726617598018901,
               'uniform_normal': 0.15350691875939732}

In [147]:
for k in BER_values:
    print((k))

gm
paired_uniforms
normal_beta
uniform_normal


In [148]:
wanted_columns = ['Dp', 'Bha',
       'Bha_knn', 'inf', 'enDive']

### Simulation stage

In [149]:
# from modules.multi_bounds_v3 import bounds_class
# # from modules.Bhatt_knn_func import knn_num_calc
# from modules.data_gen import data_gen

In [150]:
all_obj = {}

for k in BER_values:
    file_path = 'sim_data/' + k + dim_str + '.pkl'
    print(file_path, end = '\t')
    #Open the file in binary read mode
    with open(file_path, 'rb') as file:
        # Use pickle.load to deserialize and load the list of objects from the file
        loaded_objects = pickle.load(file)

    print('Loaded object successfully')
    
    all_obj[k] = loaded_objects



sim_data/gm15.pkl	Loaded object successfully
sim_data/paired_uniforms15.pkl	Loaded object successfully
sim_data/normal_beta15.pkl	Loaded object successfully
sim_data/uniform_normal15.pkl	Loaded object successfully


In [151]:
# all_obj

In [152]:
all_obj['gm']

[[<modules.multi_bounds_parfor.bounds_class at 0x7f0be8e0e650>,
 [50, 80, 125, 200, 315, 500, 800, 1250, 2000, 3150, 5000]]

### Using the data



In [153]:
# 
dpi = 250


In [154]:
validity_dict = {}

for k in all_obj:
    
    
    temp_bound_obj_lst  = all_obj[k][0]

    lst = [obj.validity(BER_values[k]) for obj in temp_bound_obj_lst ]
    df_valid = pd.DataFrame(lst, index = sample_sizes)

    good_columns  = df_valid[wanted_columns]

    short_df = pd.concat([good_columns.head(1).tail(1), good_columns.tail(1)])
    
    validity_dict[k] = short_df



In [155]:
validity_dict

{'gm':           Dp  Bha  Bha_knn  inf  enDive
 100    0.046  0.0     0.93  0.0   0.008
 10000  0.066  0.0     1.00  0.0   0.094,
 'paired_uniforms':         Dp    Bha  Bha_knn  inf  enDive
 100    0.0  0.788    0.800  0.0   0.020
 10000  0.0  1.000    0.798  0.0   0.264,
 'normal_beta':         Dp   Bha  Bha_knn  inf  enDive
 100    0.0  0.23    0.238  0.0   0.026
 10000  0.0  1.00    0.000  0.0   0.106,
 'uniform_normal':           Dp    Bha  Bha_knn  inf  enDive
 100    0.368  0.224    0.836  0.0   0.030
 10000  1.000  1.000    1.000  0.0   0.326}

In [156]:
# final_df = pd.concat(
#     [df.assign(Key=key) for key, df in validity_dict.items()],
#     ignore_index=True
# ) 

# final_df

In [157]:
#  Assuming validity_dict is populated as per your previous code
final_df_list = []

for key, df in validity_dict.items():
    # Create a new column combining the key and index
    df = df.copy()  # Avoid modifying the original DataFrame
    df['Key'] = [f'{key}_{idx}' for idx in df.index]
    
    # Append the DataFrame with the new Key column to the list
    final_df_list.append(df)

# Concatenate the list of DataFrames into one final DataFrame
final_validity = pd.concat(final_df_list, ignore_index=True)

# Display the final DataFrame
final_validity

,Dp,Bha,Bha_knn,inf,enDive,Key
0,0.046,0.000,0.930,0.0,0.008,gm_100
1,0.066,0.000,1.000,0.0,0.094,gm_10000
2,0.000,0.788,0.800,0.0,0.020,paired_uniforms_100
3,0.000,1.000,0.798,0.0,0.264,paired_uniforms_10000
4,0.000,0.230,0.238,0.0,0.026,normal_beta_100
5,0.000,1.000,0.000,0.0,0.106,normal_beta_10000
6,0.368,0.224,0.836,0.0,0.030,uniform_normal_100
7,1.000,1.000,1.000,0.0,0.326,uniform_normal_10000


In [158]:
# lst = [obj.bound_width() for obj in bound_obj_lst ]

# bound_width_df = pd.DataFrame(lst, index = sample_sizes)

# bound_width_df.head()

In [159]:
width_dict = {}

for k in all_obj:
    temp_bound_obj_lst  = all_obj[k][0]


    lst = [obj.bound_width() for obj in temp_bound_obj_lst ]
    temp_df = pd.DataFrame(lst, index = sample_sizes)

    good_columns  = temp_df[wanted_columns]

    short_df = pd.concat([good_columns.head(1).tail(1), good_columns.tail(1)])
    
    width_dict[k] = short_df




In [160]:
#  Assuming validity_dict is populated as per your previous code
final_df_list2 = []

for key, df in width_dict.items():
    # Create a new column combining the key and index
    df = df.copy()  # Avoid modifying the original DataFrame
    df['Key'] = [f'{key}_{idx}' for idx in df.index]
    
    # Append the DataFrame with the new Key column to the list
    final_df_list2.append(df)

# Concatenate the list of DataFrames into one final DataFrame
final_width = pd.concat(final_df_list2, ignore_index=True)

# Display the final DataFrame
final_width

,Dp,Bha,Bha_knn,inf,enDive,Key
0,0.115809,0.002277,0.175958,7.018095e-10,0.003427,gm_100
1,0.102750,0.199091,0.189522,9.801252e-04,0.031426,gm_10000
2,0.119645,0.179072,0.181163,6.761325e-03,0.013826,paired_uniforms_100
3,0.104576,0.153555,0.198854,7.339101e-03,0.062766,paired_uniforms_10000
4,0.115566,0.193700,0.193317,4.852327e-03,0.014024,normal_beta_100
5,0.118445,0.150002,0.206827,8.715022e-03,0.092263,normal_beta_10000
6,0.108039,0.160280,0.159646,3.244335e-03,0.018232,uniform_normal_100
7,0.101583,0.198872,0.189208,9.125538e-03,0.067237,uniform_normal_10000


In [161]:
final_validity_renamed = final_validity.rename(columns={
    'Dp': 'Dp_valid',
    'Bha': 'Bha_valid',
    'Bha_knn': 'Bha_knn_valid',
    'inf': 'inf_valid',
    'enDive': 'enDive_valid'
})

final_width_renamed = final_width.rename(columns={
    'Dp': 'Dp_width',
    'Bha': 'Bha_width',
    'Bha_knn': 'Bha_knn_width',
    'inf': 'inf_width',
    'enDive': 'enDive_width'
})

# Step 2: Merge the two DataFrames on the 'Key' column
combined_df = pd.merge(final_validity_renamed, final_width_renamed, on='Key')

# Display the combined DataFrame
columns = combined_df.columns.to_list()

columns.sort()
# combined_df.columns
columns

['Bha_knn_valid',
 'Bha_knn_width',
 'Bha_valid',
 'Bha_width',
 'Dp_valid',
 'Dp_width',
 'Key',
 'enDive_valid',
 'enDive_width',
 'inf_valid',
 'inf_width']

In [162]:
order_columns = [ 'Key',
'Bha_valid',
'Bha_width',
'Bha_knn_valid',
'Bha_knn_width',
'inf_valid',
'inf_width',
 'Dp_valid',
 'Dp_width',
 'enDive_valid',
 'enDive_width'
 ]

In [163]:
combined_sorted = combined_df[order_columns]

combined_sorted.index = combined_sorted.Key

combined_sorted= combined_sorted.drop(columns=['Key'])


combined_sorted.round(3)

,Bha_valid,Bha_width,Bha_knn_valid,Bha_knn_width,inf_valid,inf_width,Dp_valid,Dp_width,enDive_valid,enDive_width
Key,,,,,,,,,,
gm_100,0.000,0.002,0.930,0.176,0.0,0.000,0.046,0.116,0.008,0.003
gm_10000,0.000,0.199,1.000,0.190,0.0,0.001,0.066,0.103,0.094,0.031
paired_uniforms_100,0.788,0.179,0.800,0.181,0.0,0.007,0.000,0.120,0.020,0.014
paired_uniforms_10000,1.000,0.154,0.798,0.199,0.0,0.007,0.000,0.105,0.264,0.063
normal_beta_100,0.230,0.194,0.238,0.193,0.0,0.005,0.000,0.116,0.026,0.014
normal_beta_10000,1.000,0.150,0.000,0.207,0.0,0.009,0.000,0.118,0.106,0.092
uniform_normal_100,0.224,0.160,0.836,0.160,0.0,0.003,0.368,0.108,0.030,0.018
uniform_normal_10000,1.000,0.199,1.000,0.189,0.0,0.009,1.000,0.102,0.326,0.067


## MAE of bad bounds

In [164]:
MSE_dict = {}

for k in all_obj:
    
    
    temp_bound_obj_lst  = all_obj[k][0]

    lst = [obj.mae_bounds(BER_values[k]) for obj in temp_bound_obj_lst ]
    df_valid = pd.DataFrame(lst, index = sample_sizes)

    good_columns  = df_valid[wanted_columns+["tight"]]

    short_df = pd.concat([good_columns.head(1).tail(1), good_columns.tail(1)])
    
    MSE_dict[k] = short_df



In [165]:
#  Assuming MAE_dict is populated as per your previous code
final_df_list = []

for key, df in MSE_dict.items():
    # Create a new column combining the key and index
    df = df.copy()  # Avoid modifying the original DataFrame
    df['Key'] = [f'{key}_{idx}' for idx in df.index]
    
    # Append the DataFrame with the new Key column to the list
    final_df_list.append(df)

# Concatenate the list of DataFrames into one final DataFrame
final_mae = pd.concat(final_df_list, ignore_index=True)

# Display the final DataFrame
# final_mae

Conditional MAE

In [166]:
MAE_C_dict = {}

for k in all_obj:
    
    
    temp_bound_obj_lst  = all_obj[k][0]

    lst = [obj.mae_bounds_conditional(BER_values[k]) for obj in temp_bound_obj_lst ]
    df_valid = pd.DataFrame(lst, index = sample_sizes)

    good_columns  = df_valid[wanted_columns+["tight"]]

    short_df = pd.concat([good_columns.head(1).tail(1), good_columns.tail(1)])
    
    MAE_C_dict[k] = short_df



In [167]:
final_df_list = []

for key, df in MAE_C_dict.items():
    # Create a new column combining the key and index
    df = df.copy()  # Avoid modifying the original DataFrame
    df['Key'] = [f'{key}_{idx}' for idx in df.index]
    
    # Append the DataFrame with the new Key column to the list
    final_df_list.append(df)

# Concatenate the list of DataFrames into one final DataFrame
final_mae_c = pd.concat(final_df_list, ignore_index=True)

# Display the final DataFrame
# final_mae_cs

Combining the two

In [168]:

final_mae.index = final_mae.Key

final_mae= final_mae.drop(columns=['Key'])


final_mae_c.index = final_mae_c.Key

final_mae_c= final_mae_c.drop(columns=['Key'])


final_mae.columns  += "_MAE"
final_mae_c.columns  += "_MAE_C"


In [169]:
order_columns2 = ['Key', 'Bha', 'Bha_knn', 'inf', 'Dp','enDive', 'tight']

In [170]:
combined_mae = pd.concat([final_mae, final_mae_c], axis = 1)
combined_mae

,Dp_MAE,Bha_MAE,Bha_knn_MAE,inf_MAE,enDive_MAE,tight_MAE,Dp_MAE_C,Bha_MAE_C,Bha_knn_MAE_C,inf_MAE_C,enDive_MAE_C,tight_MAE_C
Key,,,,,,,,,,,,
gm_100,0.061673,0.358611,0.001419,0.138685,0.245302,0.040471,0.064647,0.358611,0.020273,0.138685,0.247280,0.041637
gm_10000,0.006031,0.064804,0.000000,0.137698,0.166436,0.011639,0.006457,0.064804,0.000000,0.137698,0.183704,0.012488
paired_uniforms_100,0.129535,0.007729,0.004826,0.093145,0.215880,0.033167,0.129535,0.036455,0.024130,0.093145,0.220286,0.034122
paired_uniforms_10000,0.049008,0.000000,0.000381,0.092606,0.045477,0.045885,0.049008,0.000000,0.001886,0.092606,0.061790,0.045885
normal_beta_100,0.204012,0.019434,0.033920,0.067758,0.221008,0.081038,0.204012,0.025239,0.044514,0.067758,0.226908,0.081200
normal_beta_10000,0.120240,0.000000,0.082392,0.063869,0.106351,0.147967,0.120240,0.000000,0.082392,0.063869,0.118961,0.147967
uniform_normal_100,0.025997,0.105154,0.003908,0.150233,0.211138,0.061530,0.041134,0.135508,0.023827,0.150233,0.217668,0.062786
uniform_normal_10000,0.000000,0.000000,0.000000,0.144297,0.050953,0.014114,0.000000,0.000000,0.000000,0.144297,0.075598,0.014142


In [171]:
order_columns2 = [
'Bha_MAE',
'Bha_MAE_C',
'Bha_knn_MAE',
'Bha_knn_MAE_C',
'inf_MAE',
'inf_MAE_C',
 'Dp_MAE',
 'Dp_MAE_C',
 'enDive_MAE',
 'enDive_MAE_C'
 ]

In [172]:
combined_mae = combined_mae[order_columns2]

combined_mae.round(4)



,Bha_MAE,Bha_MAE_C,Bha_knn_MAE,Bha_knn_MAE_C,inf_MAE,inf_MAE_C,Dp_MAE,Dp_MAE_C,enDive_MAE,enDive_MAE_C
Key,,,,,,,,,,
gm_100,0.3586,0.3586,0.0014,0.0203,0.1387,0.1387,0.0617,0.0646,0.2453,0.2473
gm_10000,0.0648,0.0648,0.0000,0.0000,0.1377,0.1377,0.0060,0.0065,0.1664,0.1837
paired_uniforms_100,0.0077,0.0365,0.0048,0.0241,0.0931,0.0931,0.1295,0.1295,0.2159,0.2203
paired_uniforms_10000,0.0000,0.0000,0.0004,0.0019,0.0926,0.0926,0.0490,0.0490,0.0455,0.0618
normal_beta_100,0.0194,0.0252,0.0339,0.0445,0.0678,0.0678,0.2040,0.2040,0.2210,0.2269
normal_beta_10000,0.0000,0.0000,0.0824,0.0824,0.0639,0.0639,0.1202,0.1202,0.1064,0.1190
uniform_normal_100,0.1052,0.1355,0.0039,0.0238,0.1502,0.1502,0.0260,0.0411,0.2111,0.2177
uniform_normal_10000,0.0000,0.0000,0.0000,0.0000,0.1443,0.1443,0.0000,0.0000,0.0510,0.0756
